# **TWEET + ACCOUNT FEATURE LSTM**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install emoji Levenshtein emot ttp wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 553.2/553.2 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.8/85.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.4/311.4 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 2.8 MB/s eta 0:00:00


In [ ]:
import os
import random
import gdown
import gensim
import gensim.downloader as gloader
import pandas as pd
import torch
from gensim.models import KeyedVectors
from sklearn.metrics import (accuracy_score, f1_score, precision_score, recall_score)
from argparse import ArgumentParser
from datetime import datetime
import pytz
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.ensemble import RandomForestClassifier
import wandb
import time
import numpy as np
import torch
import torch.nn as nn
from torch import Tensor
from tqdm import tqdm
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import json
from collections import OrderedDict, namedtuple
import torch.nn.utils.rnn as rnn
from torch.utils.data import DataLoader, Dataset
import collections
import math
import os
import re
from datetime import datetime
from statistics import mean
from string import punctuation
import emoji
import Levenshtein
import nltk
import pandas as pd
from emot.core import emot
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from pandas.core.common import flatten
from sklearn.preprocessing import StandardScaler
from ttp import ttp
from sklearn.metrics import confusion_matrix


In [ ]:
BATCH_SIZE = 128                   # number of sentences in each mini-batch
LR = 1e-3                          # learning rate
NUM_EPOCHS = 10                    # number of epochs
WEIGHT_DECAY = 1e-4                # regularization
LSTM_HIDDEN_DIM = 300              # hidden dimension of lstm network
LSTM_NUM_LAYERS = 1                # num of recurrent layers of lstm network
FREEZE = False                     # wheter to make the embedding layer trainable or not
DROPOUT = True                     # wheter to use dropout layer or not
DROPOUT_P = 0.5                    # dropout probability
EMBEDDING_MODEL_NAME = 'fastText'  # which embedding model to use
NUM_TW_FEATURES = 30               # how many tweet from the same user are exploited to compute metadata features
NUM_TW_TXT = 10
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
config = {
'batch_size' : BATCH_SIZE,
'lr' : LR,
'num_epochs' : NUM_EPOCHS,
'weight_decay' : WEIGHT_DECAY,
'lstm_hidden_dim' : LSTM_HIDDEN_DIM,
'lstm_num_layers': LSTM_NUM_LAYERS,
'freeze' : FREEZE,
'dropout' : DROPOUT,
'dropout_p' : DROPOUT_P,
'device' : DEVICE,
'emb_model_name' : EMBEDDING_MODEL_NAME,
'num_tw_features' : NUM_TW_FEATURES,
'num_tw_txt' : NUM_TW_TXT
}

In [3]:
dataset_df = pd.read_pickle('/content/drive/MyDrive/Major Project/Different combinations/data/processed_dataset_v4.pkl')

In [ ]:
dataset = pd.read_pickle('/content/drive/MyDrive/Major Project/Different combinations/data/processed_dataset_v4.pkl')

In [ ]:
emb_model_cached_path = "/content/drive/MyDrive/Major Project/Different combinations/data/twitter-multilingual-300d.new.bin"
# gdown.download(id="1DprdHGocFXJ9swnb2pDJJxHw5QR810LS",output=str(emb_model_cached_path))

In [ ]:
emb_model = KeyedVectors.load_word2vec_format(emb_model_cached_path, binary=True)
print('vectors loaded')

vectors loaded


In [5]:
text_features = ['avg_length','avg_cleaned_length','1+_mention','1+_emot','1+_url','max_hashtag','max_mentions','url_count','hashtag_count','mention_count',
    'emot_count','punct_count','rt_count','unique_hashtag_ratio','unique_rt_ratio','unique_words_ratio']

account_features = ['has_location', 'has_url', 'name_len', 'screen_name_len', 'description_len', 'followings_count','followers_count',
    'fofo_ratio', 'tweets_count', 'listed_count', 'num_in_screen_name', 'hashtag_in_description', 'def_profile', 'screen_name_entropy', 'tweet_freq', 'is_geo_enabled',
    'favourites_count', 'lev_dist_name_screeName', 'followers_growth_rate', 'followings_growth_rate', 'favourites_growth_rate']

In [ ]:
dataset_df['text_features'] = dataset_df[text_features].values.tolist()
dataset_df['account_features'] = dataset_df[account_features].values.tolist()
text_features_dim = len(text_features)
account_features_dim = len(account_features)

In [ ]:
class TweetAndMultiFeaturesDataset(Dataset):

    def __init__(self, dataframe: pd.DataFrame):
        self.tweet = dataframe['processed_tweet']
        self.label = dataframe['label']
        self.txt_features = dataframe['text_features']
        self.acc_features = dataframe['account_features']

    def __len__(self):
        return len(self.label)

    def __getitem__(self, idx):
        return {
            'tweet': self.tweet[idx],
            'label': self.label[idx],
            'txt_features': self.txt_features[idx],
            'acc_features': self.acc_features[idx]
            }

In [ ]:
train_ds = TweetAndMultiFeaturesDataset(dataset_df[dataset_df['split'] == 'train'].reset_index(drop=True))
val_ds = TweetAndMultiFeaturesDataset(dataset_df[dataset_df['split'] == 'val'].reset_index(drop=True))
test_ds = TweetAndMultiFeaturesDataset(dataset_df[dataset_df['split'] == 'test'].reset_index(drop=True))

In [ ]:
Vocab = namedtuple('Vocabulary',['word2int','int2word','unique_words'])

In [ ]:
print('Building vocab...')

unique_words : list = dataset_df['processed_tweet'].explode().unique().tolist()
unique_words.insert(0,'<pad>')
unique_words.insert(1,'<unk>')

word2int = OrderedDict()
int2word = OrderedDict()

for i, word in enumerate(unique_words):
    word2int[word] = i
    int2word[i] = word

vocab = Vocab(word2int,int2word,unique_words)

print(f'the number of unique words is {len(unique_words)}')

Building vocab...
the number of unique words is 111542


In [ ]:
print('Building embedding matrix...')

embedding_dimension = emb_model.vector_size #how many numbers each emb vector is composed of
embedding_matrix = np.zeros((len(vocab.word2int)+1, embedding_dimension), dtype=np.float32)   #create a matrix initialized with all zeros

for word, idx in vocab.word2int.items():
    if idx == 0: continue
    try:
        embedding_vector = emb_model[word]
    except (KeyError, TypeError):
        embedding_vector = np.random.uniform(low=-0.05, high=0.05, size=embedding_dimension)

    embedding_matrix[idx] = embedding_vector     #assign the retrived or the generated vector to the corresponding index

emb_matrix = embedding_matrix

print(f"Embedding matrix shape: {embedding_matrix.shape}")

Building embedding matrix...
Embedding matrix shape: (111543, 300)


In [ ]:
model_cfg = {
  'pad_idx' : vocab.word2int['<pad>'],
  'freeze_embedding' : FREEZE,
  'dropout' : DROPOUT,
  'dropout_p' : DROPOUT_P,
  'hidden_dim' : LSTM_HIDDEN_DIM,
  'num_layers': LSTM_NUM_LAYERS,
  'txt_features_dim' : text_features_dim,
  'acc_features_dim' :account_features_dim
}

In [ ]:
cfg = model_cfg
device = DEVICE

matrix = torch.from_numpy(emb_matrix).to(DEVICE)   #the embedding matrix
_ , word_embedding_dim = matrix.shape

embedding_layer=nn.Embedding.from_pretrained(matrix, freeze=cfg['freeze_embedding'], padding_idx = cfg['pad_idx'])

lstm = nn.LSTM(word_embedding_dim, cfg['hidden_dim'], num_layers = cfg['num_layers'], batch_first = True, bidirectional = True)

dropout = nn.Dropout(cfg['dropout_p'])

linear1 = nn.Linear(cfg['txt_features_dim'] + cfg['hidden_dim']*2,cfg['hidden_dim'])
linear2 = nn.Linear(cfg['acc_features_dim'] + cfg['hidden_dim'],1)

In [ ]:
train = dataset_df[dataset_df['split']=='train']
counts = train['label'].value_counts().to_dict()
print(counts)

(human, bot) = counts[0.0], counts[1.0]
weight_positive_class = torch.tensor([human/bot], device = DEVICE)  #weight to give to positive class

{1.0: 4077, 0.0: 3312}


In [ ]:
criterion = nn.BCEWithLogitsLoss(pos_weight=weight_positive_class)

In [ ]:
parameters = list(embedding_layer.parameters()) + list(lstm.parameters()) + list(dropout.parameters()) + \
              list(linear1.parameters()) + list(linear2.parameters())

In [ ]:
optimizer = optim.Adam(parameters, lr=LR, weight_decay=WEIGHT_DECAY)

In [ ]:
attr = namedtuple('attr', ['device', 'dataset', 'train_ds', 'val_ds', 'test_ds'])

In [ ]:
attr = attr(device=device, dataset=dataset_df.copy(deep=True), train_ds=train_ds, val_ds=val_ds, test_ds=test_ds)

In [ ]:
def custom_collate(batch):
    device = DEVICE

    tweet_lengths = torch.tensor([len(example['tweet']) for example in batch])

    txt_features = torch.tensor([example['txt_features'] for example in batch], device=device)
    acc_features = torch.tensor([example['acc_features'] for example in batch], device=device)

    numerized_tweets = [torch.tensor([vocab.word2int.get(token, vocab.word2int['<unk>']) for token in example['tweet']]) for example in batch]
    padded_tweets = rnn.pad_sequence(numerized_tweets, batch_first=True, padding_value=vocab.word2int['<pad>']).to(device)

    labels = torch.tensor([example['label'] for example in batch], device=device)

    return {
        'tweets': padded_tweets,
        'txt_features': txt_features,
        'acc_features': acc_features,
        'labels': labels,
        'lengths': tweet_lengths
    }


In [ ]:
# def getDataloader(split : str, batch_size : int, shuffle : bool):
#     dataset = getattr(attr,split+'_ds')
#     return DataLoader(dataset,batch_size,shuffle=shuffle,collate_fn=custom_collate)

In [ ]:
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, collate_fn=custom_collate)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, collate_fn=custom_collate)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False, collate_fn=custom_collate)

In [ ]:
test_loader

In [ ]:
test_ds

In [ ]:
def metrics(y_true, y_pred):

    acc = accuracy_score(y_true, y_pred)

    f1 = f1_score(y_true,y_pred,average='macro')

    prec = precision_score(y_true,y_pred,average='macro')

    rec = recall_score(y_true,y_pred,average="macro")

    return acc, f1, prec, rec

In [ ]:
class TweetAndAccount_model(nn.Module):

    def __init__(self, emb_matrix: np.ndarray, cfg : dict, device) :
        super().__init__()

        self.cfg = cfg
        self.device = device

        self.embedding_layer, self.word_embedding_dim = self.build_emb_layer(emb_matrix,cfg['pad_idx'], cfg['freeze_embedding'])

        self.lstm = nn.LSTM(self.word_embedding_dim, cfg['hidden_dim'], num_layers = cfg['num_layers'], batch_first = True, bidirectional = True)

        self.dropout = nn.Dropout(cfg['dropout_p'])

        self.linear1 = nn.Linear(cfg['txt_features_dim'] + cfg['hidden_dim']*2,cfg['hidden_dim'])
        self.linear2 = nn.Linear(cfg['acc_features_dim'] + cfg['hidden_dim'],1)


    def name(self):
        return 'TweetAndAccount_model'

    def build_emb_layer(self, weights_matrix: np.ndarray, pad_idx : int, freeze : bool):

        matrix = torch.from_numpy(weights_matrix).to(self.device)   #the embedding matrix
        _ , embedding_dim = matrix.shape

        emb_layer = nn.Embedding.from_pretrained(matrix, freeze=freeze, padding_idx = pad_idx)   #load pretrained weights in the layer and make it non-trainable (TODO: trainable ? )

        return emb_layer, embedding_dim


    def forward(self, batch_data):

        tweets = batch_data['tweets']           # [batch_size, num_tokens]
        tweet_lengths = batch_data['lengths']   # [batch_size]

        #embed each word in a sentence with a n-dim vector
        word_emb_tweets = self.embedding_layer(tweets)  # word_emb_tweets = [batch_size, num_tokens, embedding_dim]

        #pass the embedded tokens throught lstm network
        packed_embeddings = pack_padded_sequence(word_emb_tweets, tweet_lengths, batch_first=True, enforce_sorted=False) #tweet_lengths.cpu() TODO
        output, (hn,cn)  = self.lstm(packed_embeddings)   # hn = [2, batch_size, hidden_dim]

        #concat forward and backward output
        fwbw_hn = torch.cat((hn[-1,:,:],hn[-2,:,:]),dim=1)  # fwbw_hn = [batch_size, 2*hidden_dim]

        if self.cfg['dropout']:
            fwbw_hn = self.dropout(fwbw_hn)


        out = torch.cat([fwbw_hn,batch_data['txt_features']],dim=1)     # out = [batch_size, hidden_dim*2 + txt_features_dim]
        out = self.linear1(out)                                         # out = [batch_size, hidden_dim]
        out = F.relu(out)
        out = torch.cat([out,batch_data['acc_features']],dim=1)         # out = [batch_size, hidden_dim + acc_features_dim]
        out = self.linear2(out)                                         # out = [batch_size, 1]

        return out

In [ ]:
class Trainer():

    def __init__(self, model : nn.Module, device, criterion, optimizer) : #TODO qualcosa

        self.device = device

        model.to(self.device)
        self.model = model

        self.criterion = criterion.to(self.device) if isinstance(criterion, nn.Module) else criterion
        self.optimizer = optimizer

        # self.models_dir = glob.BASE_PATH / 'models'


    def train_loop(self, dataloader : DataLoader):

        batch_size = dataloader.batch_size
        dataset_size = len(dataloader.dataset)

        start_time = time.perf_counter()

        tot_loss = 0

        #aggregate all the predictions and corresponding true labels (and claim ids) in tensors
        all_pred , all_targ = np.empty(dataset_size), np.empty(dataset_size)

        self.model.train()

        for batch_id, batch_data in enumerate(tqdm(dataloader)):

            self.optimizer.zero_grad()

            predictions : Tensor = self.model(batch_data)   #generate predictions
            predictions = predictions.squeeze(1)

            loss = self.criterion(predictions.float(), batch_data['labels'].float())      #compute the loss

            #backward pass
            loss.backward()
            self.optimizer.step()

            pred = (predictions > 0.0 ).detach().int().cpu().numpy()           #get class label

            start = batch_id * batch_size
            end = start + batch_size

            #concatenate the new tensors with the one computed in previous steps
            all_pred[start:end] = pred
            all_targ[start:end] = batch_data['labels'].detach().cpu().numpy()

            tot_loss += loss.item()    #accumulate batch loss


        acc, f1, prec, rec = metrics(all_targ,all_pred)

        loss = tot_loss/(batch_id+1)    #mean loss

        end_time = time.perf_counter()

        return loss, acc, f1, prec, rec, end_time-start_time


    def eval_loop(self, dataloader : DataLoader):

        batch_size = dataloader.batch_size
        dataset_size = len(dataloader.dataset)

        start_time = time.perf_counter()

        tot_loss = 0

        #aggregate all the predictions and corresponding true labels (and claim ids) in tensors
        all_pred , all_targ = np.empty(dataset_size), np.empty(dataset_size)

        self.model.eval()   #model in eval mode

        with torch.no_grad(): #without computing gradients since it is evaluation loop

            for batch_id, batch_data in enumerate(tqdm(dataloader)):

                predictions : Tensor = self.model(batch_data)   #generate predictions
                predictions = predictions.squeeze(1)

                loss = self.criterion(predictions.float(), batch_data['labels'].float())      #compute the loss

                pred = (predictions > 0.0 ).detach().int().cpu().numpy()        #get class label
                start = batch_id * batch_size
                end = start + batch_size

                #concatenate the new tensors with the one computed in previous steps
                all_pred[start:end] = pred
                all_targ[start:end] = batch_data['labels'].detach().cpu().numpy()

                tot_loss += loss.item()   #accumulate batch loss

        acc, f1, prec, rec = metrics(all_targ,all_pred)

        loss = tot_loss/(batch_id+1)   #mean loss

        end_time = time.perf_counter()

        return loss, acc, f1, prec, rec, end_time-start_time


    def train_and_eval(self, train_loader, val_loader, num_epochs):
        """
            Runs the train and eval loop and keeps track of all the metrics of the training model
        """
        best_f1 = -1   #init best f1 score

        for epoch in range(1, num_epochs+1): #epoch loop

            start_time = time.perf_counter()

            print(f'Starting epoch {epoch}')

            train_metrics = self.train_loop(train_loader)
            val_metrics = self.eval_loop(val_loader)

            end_time = time.perf_counter()

            tot_epoch_time = end_time-start_time

            train_epoch_loss, train_epoch_acc, train_epoch_f1, train_epoch_prec, train_epoch_rec, train_epoch_time = train_metrics
            val_epoch_loss, val_epoch_acc, val_epoch_f1, val_epoch_prec, val_epoch_rec, val_epoch_time = val_metrics

            if val_epoch_f1 >= best_f1:
                best_f1 = val_epoch_f1
                if not os.path.exists(self.models_dir):
                    os.makedirs(self.models_dir)
                torch.save(self.
                           model.state_dict(),self.models_dir/ f'{self.model.name()}.pt')

            # wandb logs
            wandb.log({'train/loss': train_epoch_loss, 'train/acc': train_epoch_acc, 'train/f1': train_epoch_f1,
                       'train/prec': train_epoch_prec, 'train/rec': train_epoch_rec, 'train/time': train_epoch_time,
                       'val/loss': val_epoch_loss, 'val/acc': val_epoch_acc, 'val/f1': val_epoch_f1,
                       'val/prec': val_epoch_prec, 'val/rec': val_epoch_rec, 'val/time' : val_epoch_time,
                       'lr': self.optimizer.param_groups[0]['lr'], 'epoch': epoch})

            print(f'Total epoch Time: {tot_epoch_time:.4f}')
            print(f'Train Loss: {train_epoch_loss:.3f} | Train Acc: {train_epoch_acc*100:.2f}% | Train F1: {train_epoch_f1:.2f}')
            print(f'Val. Loss: {val_epoch_loss:.3f} | Val. Acc: {val_epoch_acc*100:.2f}% | Val. F1: {val_epoch_f1:.2f}')

    def test(self, test_loader):

        print('loading model state from folder')
        self.model.load_state_dict(torch.load("/content/drive/MyDrive/Major Project/Different combinations/models/TweetAndAccount_model.pt"))
        print('loaded')

        test_metrics = self.eval_loop(test_loader)
        test_epoch_loss, test_epoch_acc, test_epoch_f1, test_epoch_prec, test_epoch_rec, test_epoch_time = test_metrics

        print(f'Test -> Loss: {test_epoch_loss:.3f} | Acc: {test_epoch_acc:.3f} | F1: {test_epoch_f1:.3f} | Prec: {test_epoch_prec:.3f} | Rec: {test_epoch_rec:.3f} ')

    def predict_single_sample_from_test(self, test_loader, index):
        for batch_id, batch_data in enumerate(test_loader):
            if batch_id == index // test_loader.batch_size:
                sample_index_in_batch = index % test_loader.batch_size
                predictions = self.model(batch_data)
                predictions = predictions.squeeze(1)
                pred = (predictions > 0.0).detach().int().cpu().numpy()
                return pred[sample_index_in_batch]


In [ ]:
model = TweetAndAccount_model(emb_matrix, cfg, DEVICE)

In [ ]:
trainer = Trainer(model, device, criterion, optimizer)

In [ ]:
tweet = dataset.iloc[301]['tweet']
label = dataset.iloc[301]['label']

print(f"tweet: {tweet}, label: {label}")

tweet: @TimSeaw Yeah it is more like “we claimed it was a hotfix, but we added a submarine feature to allow us to have a public fight with Apple”
 @kumpera @RenatoFontes Haha good point!
 @rileytestut @altstoreio Some tuning there might be nice, agreed.
 @virtudude @novallkhan @github @natfriedman Also, here is my take: https://t.co/qovEKRTDYK
 @rileytestut @altstoreio I think your hoops are at the right point for mass security.
 @rileytestut @altstoreio Need an email address to share
 @RenatoFontes I also think my taxes are too high.  I should pay 0% :-)
 @virtudude @novallkhan @github @natfriedman I don’t envy Nat’s and Satya’s position.   I don’t think we should support ICE, but I don’t think those contracts can be broken without fatal wounds.  I don’t have any internal information, but I saw Novell be destroyed by shareholder pressure and that was kindergarten leagues.
 @RenatoFontes Wait until you hear what the commissions are for every good you buy every day
 @rileytestut I wrote

In [ ]:
index = 301  # Index of the sample you want to predict
predicted_label = trainer.predict_single_sample_from_test(test_loader, index)
print(f'Predicted label: {predicted_label}')

Predicted label: 1


In [ ]:
# name = datetime.now(tz = pytz.timezone('Europe/Rome')).strftime("%d/%m/%Y %H:%M:%S")
model = TweetAndAccount_model(emb_matrix, cfg, DEVICE)
trainer = Trainer(model, device, criterion, optimizer)  # Initialize the Trainer with your model, device, criterion, and optimizer
# trainer.train_and_eval(train_loader, val_loader, NUM_EPOCHS)  # Call train_and_eval method
# trainer.test(test_loader)

In [ ]:
trainer.test(test_loader)

loading model state from folder


<ipython-input-31-6ef04ede72a1>:149: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.model.load_state_dict(torch.load("/content/drive/MyDrive/Major Project/Different comb

loaded


100%|██████████| 9/9 [00:21<00:00,  2.36s/it]

Test -> Loss: 0.471 | Acc: 0.731 | F1: 0.731 | Prec: 0.736 | Rec: 0.734 


# **ENSEMBLE WITH XGBOOST**



In [6]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Train-test split
X_train = dataset_df[dataset_df['split'] == 'train'][account_features]
y_train = dataset_df[dataset_df['split'] == 'train']['label']

X_test = dataset_df[dataset_df['split'] == 'test'][account_features]
y_test = dataset_df[dataset_df['split'] == 'test']['label']

# Initialize and fit the XGBoost classifier
xgb_model = XGBClassifier(n_estimators=100, learning_rate=0.05, max_depth=6, use_label_encoder=False)
xgb_model.fit(X_train, y_train)

# Get probabilities and convert to predictions
xgb_probs = xgb_model.predict_proba(X_test)[:, 1]
xgb_preds = (xgb_probs > 0.5).astype(int)

# Calculate accuracy, precision, recall, and F1 score
accuracy = accuracy_score(y_test, xgb_preds)
precision = precision_score(y_test, xgb_preds)
recall = recall_score(y_test, xgb_preds)
f1 = f1_score(y_test, xgb_preds)

# Output the metrics
print(f"XGBoost Accuracy: {accuracy}")
print(f"XGBoost Precision: {precision}")
print(f"XGBoost Recall: {recall}")
print(f"XGBoost F1 Score: {f1}")

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [10:47:42] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBoost Accuracy: 0.778836987607245
XGBoost Precision: 0.7413010590015129
XGBoost Recall: 0.8892921960072595
XGBoost F1 Score: 0.8085808580858086


In [ ]:
def get_lstm_predictions(dataloader):
    model.eval()
    predictions = []

    with torch.no_grad():
        for batch_data in dataloader:
            output = model(batch_data)
            probs = torch.sigmoid(output).cpu().numpy()  # Get predicted probabilities
            predictions.append(probs)

    return np.concatenate(predictions)

In [ ]:
lstm_probs = get_lstm_predictions(test_loader)

In [ ]:
from sklearn.metrics import accuracy_score

lstm_probs = lstm_probs.ravel()
print(f"Flattened LSTM Probabilities shape: {lstm_probs.shape}")

print(f"XGBoost Probabilities shape: {xgb_probs.shape}")

ensemble_probs = (lstm_probs + xgb_probs) / 2  # Averaging probabilities

ensemble_preds_binary = (ensemble_probs > 0.5).astype(int)

print(f"Shape of ensemble_preds_binary: {ensemble_preds_binary.shape}")

ensemble_accuracy = accuracy_score(y_test, ensemble_preds_binary)
print(f"Ensemble Accuracy: {ensemble_accuracy}")

Flattened LSTM Probabilities shape: (1049,)
XGBoost Probabilities shape: (1049,)
Shape of ensemble_preds_binary: (1049,)
Ensemble Accuracy: 0.8064823641563393
